In [1]:
# （必要な場合だけ）依存関係
# %pip install sqlalchemy psycopg2-binary pandas

import os
import pandas as pd
from sqlalchemy import create_engine, text

# ===== 設定（環境変数で上書き可）=========================================
SCHEMA       = os.getenv("TARGET_SCHEMA", "public")
LIKE_PATTERN = os.getenv("LIKE_PATTERN", "nf_%")   # 例: "nf_%"
HEAD_N       = int(os.getenv("HEAD_N", "10"))      # 先頭プレビュー行数
MAX_ROWS_ENV = os.getenv("MAX_ROWS")               # None=全件
MAX_ROWS     = int(MAX_ROWS_ENV) if (MAX_ROWS_ENV and MAX_ROWS_ENV.isdigit()) else None
# ========================================================================

# ===== 接続文字列の決定（DATABASE_URL 優先。無ければ PGHOST 等から生成）=====
DATABASE_URL = os.getenv("DATABASE_URL")
if not DATABASE_URL:
    host = os.getenv("PGHOST", "127.0.0.1")
    port = os.getenv("PGPORT", "5432")
    db   = os.getenv("PGDATABASE", "postgres")
    user = os.getenv("PGUSER", "postgres")
    pw   = os.getenv("PGPASSWORD", "")
    DATABASE_URL = f"postgresql+psycopg2://{user}:{pw}@{host}:{port}/{db}"
# ========================================================================

# ===== エンジン作成・疎通 ==================================================
engine = create_engine(DATABASE_URL, pool_pre_ping=True)
with engine.connect() as cx:
    cx.execute(text("SELECT 1"))
print("[ok] DB接続OK")
# ========================================================================

# ===== クエリ用ヘルパ ======================================================
def q(sql: str, params=None) -> pd.DataFrame:
    """named bind だけ使う（SQLAlchemy 2.x 向け）"""
    with engine.begin() as cx:
        return pd.read_sql(text(sql), cx, params=params or {})
# ========================================================================

# ===== nf_% テーブルを列挙（存在テーブルの正式名だけ取得）==================
tables_df = q("""
SELECT table_name
FROM information_schema.tables
WHERE table_schema = :schema
  AND table_name LIKE :pat
ORDER BY table_name
""", {"schema": SCHEMA, "pat": LIKE_PATTERN})

table_names = tables_df["table_name"].tolist()
print("=== 取得対象テーブル ===")
print(table_names)
# ========================================================================

# ===== スキーマ情報（ゼロ行でもカラム定義を表示できるように先取り）=========
cols_df = q("""
SELECT table_name, ordinal_position, column_name, data_type,
       character_maximum_length, is_nullable
FROM information_schema.columns
WHERE table_schema = :schema
  AND table_name LIKE :pat
ORDER BY table_name, ordinal_position
""", {"schema": SCHEMA, "pat": LIKE_PATTERN})
# ========================================================================

# ===== 各テーブルを DataFrame 化（dfs と df_<tablename> に格納）============
dfs = {}

def load_table(schema: str, name: str, max_rows=None) -> pd.DataFrame:
    lim = f" LIMIT {int(max_rows)}" if max_rows else ""
    sql = f'SELECT * FROM "{schema}"."{name}"{lim}'
    return q(sql)

for t in table_names:
    df = load_table(SCHEMA, t, MAX_ROWS)
    dfs[t] = df
    globals()[f"df_{t}"] = df            # 例: df_nf_model_profile
    print(f"[load] {t}: {len(df)} rows × {len(df.columns)} cols")
# ========================================================================

# ===== サマリ（行数・列数）==================================================
summary_df = pd.DataFrame(
    [(t, len(dfs[t]), len(dfs[t].columns)) for t in table_names],
    columns=["table", "rows", "cols"]
).sort_values("table").reset_index(drop=True)
display(summary_df)
# ========================================================================

# ===== スキーマ＋先頭行プレビュー =========================================
pd.set_option("display.max_colwidth", 200)

def preview(name: str, head_n: int = HEAD_N, truncate: int = 200):
    print(f"\n=== {name} ===")
    # カラム定義
    schema_part = cols_df[cols_df["table_name"] == name].copy()
    display(schema_part)
    # データ
    df = dfs.get(name)
    print(f"[rows] {len(df)}")
    if len(df) > 0:
        dfv = df.copy()
        # 長文は見やすいように切り詰め
        for c in dfv.select_dtypes(include=["object"]).columns:
            dfv[c] = dfv[c].astype(str).str.slice(0, truncate)
        display(dfv.head(head_n))
    else:
        print("(empty)")

for t in table_names:
    preview(t)
# ========================================================================


[ok] DB接続OK
=== 取得対象テーブル ===
['nf_calibration', 'nf_ckpt', 'nf_config', 'nf_dataset_profile', 'nf_dm_test', 'nf_drift', 'nf_eval_interval', 'nf_eval_point', 'nf_forecasts', 'nf_hparams', 'nf_metrics', 'nf_model', 'nf_model_complexity', 'nf_model_diagnosis', 'nf_model_profile', 'nf_models', 'nf_optimization_suggestions', 'nf_parameter_sensitivity', 'nf_pkl', 'nf_predictions', 'nf_quantiles', 'nf_residual_stats', 'nf_run', 'nf_runs', 'nf_series', 'nf_training_state', 'nf_weight_statistics']
[load] nf_calibration: 0 rows × 7 cols
[load] nf_ckpt: 1 rows × 4 cols
[load] nf_config: 1 rows × 16 cols
[load] nf_dataset_profile: 0 rows × 12 cols
[load] nf_dm_test: 0 rows × 10 cols
[load] nf_drift: 0 rows × 9 cols
[load] nf_eval_interval: 0 rows × 8 cols
[load] nf_eval_point: 0 rows × 10 cols
[load] nf_forecasts: 0 rows × 10 cols
[load] nf_hparams: 1 rows × 46 cols
[load] nf_metrics: 0 rows × 8 cols
[load] nf_model: 1 rows × 62 cols
[load] nf_model_complexity: 0 rows × 7 cols
[load] nf_model_diag

,table,rows,cols
0,nf_calibration,0,7
1,nf_ckpt,1,4
2,nf_config,1,16
3,nf_dataset_profile,0,12
4,nf_dm_test,0,10
5,nf_drift,0,9
6,nf_eval_interval,0,8
7,nf_eval_point,0,10
8,nf_forecasts,0,10
9,nf_hparams,1,46



=== nf_calibration ===


,table_name,ordinal_position,column_name,data_type,character_maximum_length,is_nullable
0,nf_calibration,1,cal_id,bigint,NaN,NO
1,nf_calibration,2,run_id,bigint,NaN,YES
2,nf_calibration,3,model_id,bigint,NaN,YES
3,nf_calibration,4,level,integer,NaN,YES
4,nf_calibration,5,coverage,double precision,NaN,YES
5,nf_calibration,6,mil,double precision,NaN,YES
6,nf_calibration,7,winkler,double precision,NaN,YES


[rows] 0
(empty)

=== nf_ckpt ===


...

[rows] 0
(empty)

=== nf_training_state ===


,table_name,ordinal_position,column_name,data_type,character_maximum_length,is_nullable
304,nf_training_state,1,id,integer,NaN,NO
305,nf_training_state,2,model_dir_hash,character varying,64.0,YES
306,nf_training_state,3,epoch,integer,NaN,YES
307,nf_training_state,4,global_step,integer,NaN,YES
308,nf_training_state,5,early_stopped,boolean,NaN,YES
309,nf_training_state,6,final_lr,real,NaN,YES
310,nf_training_state,7,checkpoint_size_mb,real,NaN,YES
311,nf_training_state,8,optimizer_state,jsonb,NaN,YES


[rows] 0
(empty)

=== nf_weight_statistics ===


,table_name,ordinal_position,column_name,data_type,character_maximum_length,is_nullable
312,nf_weight_statistics,1,id,integer,NaN,NO
313,nf_weight_statistics,2,model_dir_hash,character varying,64.0,YES
314,nf_weight_statistics,3,layer_name,character varying,200.0,YES
315,nf_weight_statistics,4,layer_type,character varying,50.0,YES
316,nf_weight_statistics,5,param_count,integer,NaN,YES
317,nf_weight_statistics,6,mean_val,real,NaN,YES
318,nf_weight_statistics,7,std_val,real,NaN,YES
319,nf_weight_statistics,8,min_val,real,NaN,YES
320,nf_weight_statistics,9,max_val,real,NaN,YES
321,nf_weight_statistics,10,l1_norm,real,NaN,YES


[rows] 0
(empty)
